# Test MISA CRM API Endpoints

Notebook này dùng để kiểm tra việc lấy dữ liệu từ các API endpoint của MISA CRM. 

**Yêu cầu:** Cần điền đầy đủ `MISA_CRM_CLIENT_ID` và `MISA_CRM_CLIENT_SECRET` trong file `.env` trước khi chạy.

In [1]:
import requests
import os
import json
import pandas as pd
from dotenv import load_dotenv

# Tải các biến môi trường từ file .env
load_dotenv()

# Lấy thông tin từ biến môi trường
MISA_CLIENT_ID = os.getenv('MISA_CRM_CLIENT_ID')
MISA_CLIENT_SECRET = os.getenv('MISA_CRM_CLIENT_SECRET')
BASE_URL = 'https://crmconnect.misa.vn'

print(f"Client ID: {MISA_CLIENT_ID}")
print(f"Client Secret is {'set' if MISA_CLIENT_SECRET else 'not set'}")

Client ID: FACOLOS
Client Secret is set


## 1. Lấy Access Token

In [2]:
def get_misa_access_token():
    """Hàm để lấy access token từ MISA CRM API."""
    token_url = f"{BASE_URL}/api/v2/Account"
    
    if not MISA_CLIENT_ID or not MISA_CLIENT_SECRET:
        print('Lỗi: Vui lòng điền MISA_CRM_CLIENT_ID và MISA_CRM_CLIENT_SECRET trong file .env')
        return None
    
    payload = {
        'client_id': MISA_CLIENT_ID,
        'client_secret': MISA_CLIENT_SECRET
    }
    headers = {'Content-Type': 'application/json'}
    
    try:
        response = requests.post(token_url, json=payload, headers=headers, timeout=30)
        response.raise_for_status()
        
        result = response.json()
        if result.get('success') and result.get('data'):
            access_token = result['data']
            print('Lấy access token thành công!')
            return access_token
        else:
            print(f"Lấy token thất bại: {result.get('error_message', result)}")
            return None
    except requests.exceptions.RequestException as e:
        print(f'Lỗi kết nối khi lấy token: {e}')
        return None

access_token = get_misa_access_token()
# In một phần token để xác nhận
if access_token:
    print(f'Access Token (Full): {access_token}')

Lấy access token thành công!
Access Token (Full): eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJwYXlsb2FkZGF0YSI6Ik9NdVc2dytKL0o4dWN0ZUVwQ2x4dkw0aHFzM2VzekVna3J6Yms3ZmxCUVZWYnBwTVZDczNtMWxRRC9DcGRHNnhJTnkrL2tnZ3N1Y3hjdWRwZU9rTDRENTJKakxIN1lRM2IyV0lKVW54ZnE3Y3RIRTUzK3hoT0k3TE4wRU15S0NlSEFZbU0zK0xLUzlVdkYyWjdFZ21lMW00VDFOOXpkdUMvay8rRkRKaXhia1NGcVAyQ21FVUlxeEwzTExhei9vK3R6SXJzdThKOEFpaDlmUjZsWWtjZWZCdXZZZ3NWL3pSUmJvOE1tZUxMYSs2RzhlT21PT3Fta2J3TjFYdzhHTmJFN3JmYXJScXhpeWJGZnZIRTg3TklFSEt3QmRHa2c5Q2ZjTWZGa1dvWGtMODk5TkdBb2FuZjRNaWcrL2ZLaUtLTTVTV3dvUUZHOTVzTG15YnhXemJHQT09IiwiZXhwIjoxNzYxNzI4MTU3LCJpc3MiOiJNSVNBIiwiYXVkIjoiQU1JU0NSTTIifQ.3V_xqNXw-snErRrpVIvLgMuQFOXdfgglDpWiR8dnhbE


## 2. Hàm Test Endpoint

In [3]:
def test_endpoint(endpoint_path, access_token, params=None):
    """Hàm chung để test một endpoint với phân trang."""
    if not access_token:
        print('Không có access token, không thể thực hiện request.')
        return
    
    url = f"{BASE_URL}{endpoint_path}"
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Clientid': MISA_CLIENT_ID
    }
    
    # Mặc định lấy trang đầu tiên, 5 bản ghi
    default_params = {'page': 0, 'pageSize': 5}
    if params:
        default_params.update(params)
    
    print(f'--- Đang gọi đến: {url} với params: {default_params} ---')
    
    try:
        response = requests.get(url, headers=headers, params=default_params, timeout=30)
        response.raise_for_status()
        
        result = response.json()
        print(f'Status Code: {response.status_code}')
        print('Response JSON:')
        print(json.dumps(result, indent=2, ensure_ascii=False))
        
        if result.get('success') and isinstance(result.get('data'), list):
            df = pd.DataFrame(result['data'])
            print('Hiển thị dữ liệu dưới dạng DataFrame:')
            display(df)
        else:
            print('Không có dữ liệu hoặc cấu trúc response không đúng.')
            
    except requests.exceptions.RequestException as e:
        print(f'Lỗi request: {e}')

## 3. Test Các Endpoint Cụ Thể

In [7]:
# Endpoint 1: Customers
test_endpoint('/api/v2/Customers', access_token)

--- Đang gọi đến: https://crmconnect.misa.vn/api/v2/Customers với params: {'page': 0, 'pageSize': 5} ---
Status Code: 200
Response JSON:
{
  "success": true,
  "code": 200,
  "data": [
    {
      "id": 672,
      "owner_name": "Nguyễn Việt Hưng (NV10)",
      "account_short_name": null,
      "fax": null,
      "parent_account_name": null,
      "account_type": "NPP",
      "sector_name": null,
      "no_of_employee_name": null,
      "bank_account": null,
      "celebrate_date": null,
      "lead_source": "Nhân viên kinh doanh tự tìm kiếm",
      "created_date": "2025-02-03T09:56:08.000+07:00",
      "created_by": "Nguyễn Việt Hưng (NV10)",
      "account_name": "Hộ kinh doanh Nguyễn Thị Thanh Vân",
      "rating": null,
      "office_email": null,
      "website": null,
      "account_number": "NPP02",
      "business_type": null,
      "industry": null,
      "annual_revenue": null,
      "budget_code": null,
      "bank_name": null,
      "customer_since_date": null,
      "is_pub

,id,owner_name,account_short_name,fax,parent_account_name,account_type,sector_name,no_of_employee_name,bank_account,celebrate_date,...,purchase_date_first,is_portal_access,portal_username,shipping_long,shipping_lat,billing_long,billing_lat,custom_field13,custom_field14,list_product_name
0,672,Nguyễn Việt Hưng (NV10),None,None,None,NPP,None,None,None,None,...,2025-02-04T00:00:00.000+07:00,False,None,None,None,105.82021160277432,21.004613446629737,None,None,"Backpack Elite Facolos - Blue,Backpack Elite F..."
1,3895,Dư Hoàng Anh (NV07),None,None,None,BANLE,None,None,None,None,...,2025-10-16T00:00:00.000+07:00,None,None,None,None,None,None,None,None,Paddles Pro Series - Drive Swift Standard- 16M...
2,3890,Đoàn Đức Trung (DOANTRUNG),Nguyễn Thị huế,None,None,NPP,None,None,None,None,...,2025-10-16T00:00:00.000+07:00,None,None,None,None,None,None,None,None,F-Pro Performance Pickleball Gen2 ( Box 150 Ba...
3,3894,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,3893,Lê Anh Tuấn (NV11),None,None,None,CHUSAN,None,None,None,None,...,2025-10-16T00:00:00.000+07:00,None,None,None,None,None,None,None,None,F-Pro Performance Pickleball Gen2


In [4]:
# Endpoint 2: SaleOrders
test_endpoint('/api/v2/SaleOrders', access_token)

--- Đang gọi đến: https://crmconnect.misa.vn/api/v2/SaleOrders với params: {'page': 0, 'pageSize': 5} ---
Status Code: 200
Response JSON:
{
  "success": true,
  "code": 200,
  "data": [
    {
      "id": 6007,
      "sale_order_no": "DH181020267",
      "campaign_name": null,
      "book_date": "2025-10-18T00:00:00.000+07:00",
      "sale_order_amount": 700000.0,
      "deadline_date": null,
      "quote_name": null,
      "status": "Chưa thực hiện",
      "exchange_rate": 1.0,
      "sale_order_name": "MKT ĐỐI NGOẠI - TRẢ THƯỞNG CLB ROYAL HUẾ T9 - HÀNG TẶNG KHÔNG THU TIỀN",
      "sale_order_date": "2025-10-18T00:00:00.000+07:00",
      "account_name": "Từ Xuân Cảnh",
      "contact_name": null,
      "due_date": "2025-10-18T00:00:00.000+07:00",
      "delivery_status": null,
      "sale_order_type": "Bán mới",
      "opportunity_name": null,
      "revenue_status": "Đề nghị ghi",
      "currency_type": "VND",
      "description": null,
      "balance_receipt_amount": 700000.0,
      

,id,sale_order_no,campaign_name,book_date,sale_order_amount,deadline_date,quote_name,status,exchange_rate,sale_order_name,...,production_date,production_rejection_reason,produced_quantity_summary,account_code,contact_code,opportunity_code,quote_code,employee_code,is_deleted,sale_order_product_mappings
0,6007,DH181020267,None,2025-10-18T00:00:00.000+07:00,7.000000e+05,None,None,Chưa thực hiện,1.0,MKT ĐỐI NGOẠI - TRẢ THƯỞNG CLB ROYAL HUẾ T9 - ...,...,None,None,0.0,KH20092268,None,None,None,HUYENTL,False,"[{'id': 22343, 'to_currency_oc': 648148.15, 'd..."
1,5597,DH101020448,None,2025-10-16T00:00:00.000+07:00,1.953704e+08,None,None,Đang thực hiện,26154.0,"Đơn hàng bán cho PICKLEBALL ONE, INC",...,None,None,0.0,KH00001141,None,None,None,DOTRUNGNGHIA,False,"[{'id': 20981, 'to_currency_oc': 633.0, 'disco..."
2,6006,DH181020266,None,2025-10-18T00:00:00.000+07:00,7.000000e+05,None,None,Chưa thực hiện,1.0,MKT ĐỐI NGOẠI- TRẢ THƯỞNG CLB ROYAL HUẾ T9- HÀ...,...,None,None,0.0,KH00001376,None,None,None,HUYENTL,False,"[{'id': 22342, 'to_currency_oc': 648148.15, 'd..."
3,5836,DH101020773,None,2025-10-18T00:00:00.000+07:00,4.533935e+07,None,None,Đang thực hiện,1.0,Đơn hàng bán cho Chị Vân ĐN,...,None,None,0.0,KH20092226,None,None,None,HOANGPHAN,False,"[{'id': 21862, 'to_currency_oc': 8333333.34, '..."
4,6005,DH181020265,None,2025-10-18T00:00:00.000+07:00,7.000000e+05,None,None,Chưa thực hiện,1.0,MKT ĐỐI NGOẠI - TRẢ THƯỞNG CLB ROYAL HUẾ T9 - ...,...,None,None,0.0,KH20092267,None,None,None,HUYENTL,False,"[{'id': 22341, 'to_currency_oc': 648148.15, 'd..."


In [9]:
# Endpoint 3: Contacts
test_endpoint('/api/v2/Contacts', access_token)

--- Đang gọi đến: https://crmconnect.misa.vn/api/v2/Contacts với params: {'page': 0, 'pageSize': 5} ---
Status Code: 200
Response JSON:
{
  "success": true,
  "code": 200,
  "data": [
    {
      "id": 6913,
      "contact_code": "LH0000229",
      "salutation": null,
      "last_name": "Thu Thảo",
      "first_name": null,
      "contact_name": "Thu Thảo",
      "title": null,
      "department": null,
      "account_name": null,
      "mobile": "0947342259",
      "other_phone": null,
      "office_email": null,
      "email": null,
      "office_tel": null,
      "lead_source": null,
      "account_type": null,
      "customer_since_date": null,
      "description": null,
      "email_opt_out": null,
      "phone_opt_out": null,
      "mailing_country": null,
      "mailing_province": null,
      "mailing_district": null,
      "mailing_ward": null,
      "mailing_street": null,
      "mailing_zip": null,
      "mailing_address": null,
      "shipping_country": null,
      "shipping

,id,contact_code,salutation,last_name,first_name,contact_name,title,department,account_name,mobile,...,last_interaction_date,last_visit_date,number_days_not_interacted,last_call_date,related_users,mailing_long,mailing_lat,shipping_long,shipping_lat,account_code
0,6913,LH0000229,None,Thu Thảo,None,Thu Thảo,None,None,None,0947342259,...,None,None,None,None,None,None,None,None,None,None
1,6901,LH0000217,None,Jun Hwa,Joe Lew,Joe Lew Jun Hwa,None,None,DINKit Store Sdn Bhd,+(60)173997933,...,None,None,None,None,None,None,None,None,None,KH00000336
2,6912,LH0000228,None,Anh Tùng,None,Anh Tùng,None,None,Wouter Cornelis van der Wiel,0938 242 756,...,None,None,None,None,None,None,None,None,None,KH00001299
3,6911,LH0000227,None,Mr. Tony Truong,None,Mr. Tony Truong,None,None,Mr Tony Truong,+8613922274686,...,None,None,None,None,None,None,None,None,None,KH00001279
4,6910,LH0000226,None,Phương,Vũ Thu,Vũ Thu Phương,None,None,None,0963457295,...,None,None,None,None,None,None,None,None,None,None


In [10]:
# Endpoint 4: Stocks
# Endpoint này không có phân trang, nên không cần params
test_endpoint('/api/v2/Stocks', access_token, params={})

--- Đang gọi đến: https://crmconnect.misa.vn/api/v2/Stocks với params: {'page': 0, 'pageSize': 5} ---
Status Code: 200
Response JSON:
{
  "success": true,
  "code": 0,
  "data": [
    {
      "act_database_id": "2c8f5040-9ccf-4fd6-882b-74f0390275ae",
      "async_id": "9d3a5603-bd40-4283-b3a4-f311b8804a92",
      "created_by": "SME",
      "created_date": "2025-03-06T20:42:09.000+07:00",
      "inactive": false,
      "modified_by": "SME",
      "modified_date": "2025-03-06T20:42:09.000+07:00",
      "stock_code": "CCDC",
      "stock_name": "Công cụ dụng cụ"
    },
    {
      "act_database_id": "2c8f5040-9ccf-4fd6-882b-74f0390275ae",
      "async_id": "8a8c5814-ecbe-4da2-851d-a84d9a808c5a",
      "created_by": "SME - CRM",
      "created_date": "2025-09-17T14:28:52.000+07:00",
      "inactive": false,
      "modified_by": "SME - CRM",
      "modified_date": "2025-09-17T14:28:52.000+07:00",
      "stock_code": "FCL HOAN_HUY SÀN",
      "stock_name": "Kho hàng hoàn, hủy sàn chưa xử lý"

,act_database_id,async_id,created_by,created_date,inactive,modified_by,modified_date,stock_code,stock_name,description
0,2c8f5040-9ccf-4fd6-882b-74f0390275ae,9d3a5603-bd40-4283-b3a4-f311b8804a92,SME,2025-03-06T20:42:09.000+07:00,False,SME,2025-03-06T20:42:09.000+07:00,CCDC,Công cụ dụng cụ,NaN
1,2c8f5040-9ccf-4fd6-882b-74f0390275ae,8a8c5814-ecbe-4da2-851d-a84d9a808c5a,SME - CRM,2025-09-17T14:28:52.000+07:00,False,SME - CRM,2025-09-17T14:28:52.000+07:00,FCL HOAN_HUY SÀN,"Kho hàng hoàn, hủy sàn chưa xử lý",NaN
2,2c8f5040-9ccf-4fd6-882b-74f0390275ae,88f5a4d5-6d04-4453-b442-9926e6978908,SME,2025-03-06T20:31:11.000+07:00,False,SME,2025-03-06T20:42:09.000+07:00,FCL_KHO_ T,Kho T,NaN
3,2c8f5040-9ccf-4fd6-882b-74f0390275ae,5cd50a16-c112-41bd-bc19-41dd45c33861,SME - CRM,2025-08-22T10:53:06.000+07:00,False,SME - CRM,2025-08-22T10:53:06.000+07:00,FCL_KHO_ VP,Kho văn phòng,NaN
4,2c8f5040-9ccf-4fd6-882b-74f0390275ae,8b539b98-a919-4263-9457-3af46dd1a2b0,SME,2025-02-05T12:35:50.000+07:00,False,SME,2025-03-06T20:42:09.000+07:00,FCL_KHO_HCM,Kho Facolos Hồ Chí Minh,NaN
5,2c8f5040-9ccf-4fd6-882b-74f0390275ae,eee3bd2d-4642-4621-ba80-52bf98786e23,SME,2025-06-04T11:31:23.000+07:00,False,SME,2025-06-04T11:31:23.000+07:00,FCL_KHO_HN_DVAO,Kho Facolos Hà Nội - kho đầu vào,Hà Nội
6,2c8f5040-9ccf-4fd6-882b-74f0390275ae,37a1ad14-675d-4685-a336-3a0fbb958d7e,SME,2025-03-06T20:42:09.000+07:00,False,SME,2025-03-06T20:42:09.000+07:00,FCL_KHO_HN_HT,Kho HT,NaN
7,2c8f5040-9ccf-4fd6-882b-74f0390275ae,7b2b90e4-4a3a-4316-a3b3-632e42066610,SME,2025-01-16T16:21:40.000+07:00,False,SME,2025-03-06T20:42:09.000+07:00,FCL_KHO_HN_LE,Kho Facolos Hà Nội - Bán Lẻ,NaN
8,2c8f5040-9ccf-4fd6-882b-74f0390275ae,6a2c42de-1fef-4b15-8af8-b4eb00cf9840,SME,2025-03-06T20:42:09.000+07:00,False,SME,2025-03-06T20:42:09.000+07:00,FCL_KHO_HN_LOI,Kho Facolos Hà Nội - Hàng Lỗi,NaN
9,2c8f5040-9ccf-4fd6-882b-74f0390275ae,7942a5fc-1b31-4f66-890a-79fbb20b3444,SME,2025-03-06T20:42:09.000+07:00,False,SME,2025-03-06T20:42:09.000+07:00,FCL_KHO_HN_SI,Kho Facolos Hà Nội - Bán Sỉ,NaN


In [11]:
# Endpoint 5: Products (Giả định, vì không có trong tài liệu v2)
# Chúng ta sẽ thử gọi đến /api/v2/Products để xem kết quả
print('Lưu ý: Endpoint /api/v2/Products không được liệt kê trong tài liệu Swagger v2, kết quả có thể không thành công.')
test_endpoint('/api/v2/Products', access_token)

Lưu ý: Endpoint /api/v2/Products không được liệt kê trong tài liệu Swagger v2, kết quả có thể không thành công.
--- Đang gọi đến: https://crmconnect.misa.vn/api/v2/Products với params: {'page': 0, 'pageSize': 5} ---
Status Code: 200
Response JSON:
{
  "success": true,
  "code": 200,
  "data": [
    {
      "id": 15400,
      "owner_name": null,
      "product_code": "BOXTRAINING150BALLS",
      "product_category": "Balls",
      "usage_unit": "Thùng",
      "product_name": "F-Train ( Box 150 Balls)",
      "inactive": false,
      "unit_price": 4861111.0,
      "tax": "8%",
      "description": null,
      "created_date": "2025-09-18T17:10:07.000+07:00",
      "created_by": "SME",
      "modified_date": "2025-10-15T17:49:32.000+07:00",
      "modified_by": "SME",
      "tag": null,
      "purchased_price": 0.0,
      "price_after_tax": false,
      "is_public": null,
      "form_layout": "Mẫu tiêu chuẩn",
      "organization_unit_name": "CÔNG TY CỔ PHẦN THỂ THAO FACOLOS",
      "is_fol

,id,owner_name,product_code,product_category,usage_unit,product_name,inactive,unit_price,tax,description,...,sale_description,quantity_formula,product_properties,default_stock,warranty_period,source,is_set_product,warranty_description,avatar,is_no_recorded_sales
0,15400,None,BOXTRAINING150BALLS,Balls,Thùng,F-Train ( Box 150 Balls),False,4861111.0,8%,None,...,"Bóng Pickleball không in, màu vàng ne-on",None,Thành phẩm,None,None,None,False,None,None,None
1,15465,None,PPS-EliteXEVN-16MM-STN,Pro Series,Chiếc,Paddles Pro Series - EliteX Elongate- 16MM - T...,False,4629630.0,8%,None,...,"Vợt Pickleball Paddles, Pro Series EliteX Elon...",None,Thành phẩm,FCL_KHO_HN_LE,None,None,False,None,None,None
2,15471,None,DIEUHOA,Hàng hóa,Chiếc,Điều Hòa 12000,False,0.0,None,None,...,Điều Hòa 12000,None,Hàng hóa,None,None,None,False,None,None,None
3,15470,None,MAYTINH,Hàng hóa,Bộ,Máy Tính,False,0.0,None,None,...,Máy Tính,None,Hàng hóa,None,None,None,False,None,None,None
4,15469,None,TP Link,Hàng hóa,Cái,Thiết bị chia mạng - TP Link,False,0.0,None,None,...,Thiết bị chia mạng - TP Link,None,Hàng hóa,None,None,None,False,None,None,None
